In [1]:
# %%capture
# !pip install torch-geometric
# !pip install py4cytoscape seaborn
# !pip install scikit-learn #dim reduction

In [2]:
#_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.7.0' # optional
import requests
import py4cytoscape as p4c
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Loading Javascript client ... 2bad58b3-61ac-4b79-81e6-54c251f39a3b on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

In [3]:
# Get Cytoscape version information
print(p4c.cytoscape_version_info())

{'apiVersion': 'v1', 'cytoscapeVersion': '3.10.2', 'automationAPIVersion': '1.11.0', 'py4cytoscapeVersion': '1.11.0'}


In [4]:
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
import py4cytoscape as p4c
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image as IPyImage
import networkx as nx
import time
from torch.nn import CrossEntropyLoss

In [5]:
# 1. Data Loading và Preprocessing
data_path = r"C:\Users\ADMIN\Downloads\ChCh-Miner_durgbank-chem-chem (1).tsv"
df = pd.read_csv(data_path, sep='\t', names=["source", "target"], header=None)
# Display the DataFrame to verify
print("DataFrame head:")
print(df.head())

DataFrame head:
    source   target
0  DB00862  DB00966
1  DB00575  DB00806
2  DB01242  DB08893
3  DB01151  DB08883
4  DB01235  DB01275


In [ ]:
# Read the TSV file with no header and assign column names 'source' and 'target'
df = pd.read_csv(data_path, sep='\t', names=["source", "target"], header=None)

# Display the DataFrame to verify
print("DataFrame head:")
print(df.head())

# Tạo các danh sách các node duy nhất (source và target)
unique_sources = df['source'].unique()
unique_targets = df['target'].unique()
unique_nodes = np.unique(np.concatenate((unique_sources, unique_targets)))

# Tạo các mappings giữa node và chỉ số
node_to_idx = {node: idx for idx, node in enumerate(unique_nodes)}
idx_to_node = {idx: node for node, idx in node_to_idx.items()}

# Tổng số node
num_nodes = len(unique_nodes)
print(f'Total nodes: {num_nodes}')

# Tạo đồ thị đồng nhất bằng NetworkX
G = nx.Graph()

# Thêm các node
for node in unique_nodes:
    G.add_node(node)

# Thêm các edge
edge_list = df[['source', 'target']].values.tolist()
G.add_edges_from(edge_list)

# Kiểm tra số lượng node và edge
print(f'Number of nodes: {G.number_of_nodes()}')
print(f'Number of edges: {G.number_of_edges()}')

# Tạo danh sách các edge dưới dạng chỉ số
edge_index = torch.tensor([
    [node_to_idx[source], node_to_idx[target]] for source, target in df.values
], dtype=torch.long).t().contiguous()

# Tạo node features (giả sử không có thuộc tính, sử dụng embedding ngẫu nhiên)
hidden_dim = 64
x = torch.randn(num_nodes, hidden_dim)  # Sử dụng embedding ngẫu nhiên

# Tạo nhãn cho node (có thể sử dụng nhãn ngẫu nhiên hoặc dựa trên loại node)
# Ví dụ với nhãn ngẫu nhiên:
y = torch.randint(0, 2, (num_nodes,), dtype=torch.long)

# Nếu muốn phân loại dựa trên loại node (source vs target), sử dụng đoạn mã sau:
# labels = []
# for node in unique_nodes:
#     if node in unique_sources:
#         labels.append(0)  # Label 0 cho source nodes
#     else:
#         labels.append(1)  # Label 1 cho target nodes
# y = torch.tensor(labels, dtype=torch.long)

# Tạo đối tượng Data
data = Data(x=x, edge_index=edge_index, y=y)

# Chia dữ liệu thành train/val/test cho node classification
train_idx, temp_idx = train_test_split(range(num_nodes), train_size=0.7, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

# Tạo masks
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

# Thêm masks vào đối tượng data
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

# Model Definition
class HomogeneousGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(HomogeneousGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.node_classifier = torch.nn.Linear(out_channels, 2)  # Đối với phân loại nhãn nhị phân

    def forward(self, data):
        x, edge_index = data.x, data.edge_index #Shape: (N, in_channels)
        # GNN Layers
        x = self.conv1(x, edge_index)
        x = F.relu(x) #Shape: (N, hidden_channels)
        x = self.conv2(x, edge_index) #Shape: (N, out_channels)
        # Node Classification
        out = self.node_classifier(x) #Shape: (N, 2)
        return out, x  # Trả về cả dự đoán và embedding vectors

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, mask, criterion=None):
    model.eval()
    out, embeddings = model(data)
    if criterion:
        loss = criterion(out[mask], data.y[mask]).item()
    pred = out[mask].argmax(dim=1)
    correct = (pred == data.y[mask]).sum().item()
    acc = correct / mask.sum().item()
    return acc

print("Setting up training...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HomogeneousGNN(in_channels=hidden_dim, hidden_channels=128, out_channels=64).to(device)
data = data.to(device)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

print("Starting training...")
best_val_acc = 0
best_epoch = 0

for epoch in range(1, 501):
    loss = train(model, data, optimizer, criterion)
    if epoch % 20 == 0:
        train_acc = evaluate(model, data, data.train_mask)
        val_acc = evaluate(model, data, data.val_mask)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pt')

# Đánh giá trên bộ test
print("\nEvaluating final model...")
model.load_state_dict(torch.load('best_model.pt'))
test_acc = evaluate(model, data, data.test_mask)
print(f'\nTest Accuracy: {test_acc:.4f}')

print("\nGenerating embeddings...")
model.eval()
with torch.no_grad():
    _, embeddings = model(data)
    embeddings = embeddings.cpu().numpy()

# Áp dụng PCA -> giảm chiều
print("\nApplying PCA for dimensionality reduction...")
pca = PCA(n_components=2, random_state=42)
embeddings_pca = pca.fit_transform(embeddings)

# Áp dụng t-SNE
print("Applying t-SNE for dimensionality reduction...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
embeddings_tsne = tsne.fit_transform(embeddings)

# Chuẩn hóa các thành phần t-SNE để trực quan hóa Cytoscape
scaler = MinMaxScaler()
embeddings_tsne_norm = scaler.fit_transform(embeddings_tsne)

# Tính độ lớn của các thành phần t-SNE
tsne_magnitude = np.linalg.norm(embeddings_tsne_norm, axis=1)

# Tạo DataFrame cho embeddings
embeddings_df = pd.DataFrame(embeddings, columns=[f'embedding_{i}' for i in range(embeddings.shape[1])])
embeddings_df['name'] = [idx_to_node[idx] for idx in range(num_nodes)]
embeddings_df['pca_0'] = embeddings_pca[:, 0]
embeddings_df['pca_1'] = embeddings_pca[:, 1]
embeddings_df['tsne_0'] = embeddings_tsne[:, 0]
embeddings_df['tsne_1'] = embeddings_tsne[:, 1]
embeddings_df['tsne_magnitude'] = tsne_magnitude
embeddings_df['label'] = y.cpu().numpy()

# 17. Chuyển Đổi edge_list_tuples Thành Chỉ Số Node
print("\nConverting edge_list_tuples to node indices...")
edge_list_tuples = [(node_to_idx[u], node_to_idx[v]) for u, v in edge_list]

# 18. Chia Cạnh Thành Train/Validation/Test Cho Link Prediction
print("\nSplitting edges into train/validation/test sets for Link Prediction...")
train_edges, test_edges = train_test_split(edge_list_tuples, test_size=0.2, random_state=42)
val_edges, test_edges = train_test_split(test_edges, test_size=0.5, random_state=42)

# 19. Tạo Negative Edges Cho Tập Train
print("\nCreating negative edges for training...")
num_negative_samples = len(train_edges)
negative_edges = []
while len(negative_edges) < num_negative_samples:
    u, v = np.random.choice(num_nodes, size=2, replace=False)
    if (u, v) not in G.edges() and (v, u) not in G.edges():  # Đảm bảo cạnh âm không tồn tại
        negative_edges.append((u, v))

import torch.nn as nn
# 20. Định Nghĩa Link Predictor Model
print("\nDefining the Link Predictor model...")
class LinkPredictor(nn.Module):
    def __init__(self, embedding_dim):
        super(LinkPredictor, self).__init__()
        self.linear = nn.Linear(embedding_dim * 2, 1)  # Cho phép nối embeddings

    def forward(self, node1_embedding, node2_embedding):
        combined_embedding = torch.cat([node1_embedding, node2_embedding], dim=1)
        output = torch.sigmoid(self.linear(combined_embedding))  # Dự đoán xác suất liên kết
        return output

# 21. Tạo Link Predictor và Định Nghĩa Optimizer & Criterion
print("\nSetting up Link Predictor training...")
embedding_dim = embeddings.shape[1]
link_predictor = LinkPredictor(embedding_dim).to(device)
optimizer_lp = torch.optim.Adam(link_predictor.parameters(), lr=0.01)
criterion_lp = nn.BCELoss()

# 22. Định Nghĩa Hàm Để Lấy Embeddings Cho Các Cạnh
def get_edge_embeddings(edges, embeddings):
    # Tách các chỉ số nguồn và đích từ danh sách cạnh
    src = [edge[0] for edge in edges]
    dst = [edge[1] for edge in edges]
    src = np.array(src)
    dst = np.array(dst)
    src_embeddings = embeddings[src]
    dst_embeddings = embeddings[dst]
    return src_embeddings, dst_embeddings

# 23. Định Nghĩa Hàm Để Tạo Labels Cho Link Prediction
def get_link_labels(positive_edges, negative_edges):
    # Tạo labels: 1 cho positive edges, 0 cho negative edges
    labels = torch.cat([torch.ones(len(positive_edges)), torch.zeros(len(negative_edges))], dim=0).to(device)
    return labels

# 24. Định Nghĩa Hàm Để Đánh Giá Link Prediction
@torch.no_grad()
def evaluate_link_prediction(model, link_predictor, edges, embeddings, criterion):
    model.eval()  # Đảm bảo mô hình GNN không được cập nhật trong quá trình đánh giá
    link_predictor.eval()
    src_embeddings, dst_embeddings = get_edge_embeddings(edges, embeddings)
    src_embeddings = torch.tensor(src_embeddings, dtype=torch.float32).to(device)
    dst_embeddings = torch.tensor(dst_embeddings, dtype=torch.float32).to(device)
    predictions = link_predictor(src_embeddings, dst_embeddings).squeeze()
    loss = criterion(predictions, torch.ones_like(predictions)).item()
    return loss

# 25. Định Nghĩa Hàm Huấn Luyện Link Predictor
def train_link_prediction(model, link_predictor, optimizer, criterion, train_edges, negative_edges, embeddings):
    model.eval()  # Đảm bảo mô hình GNN không được cập nhật trong quá trình huấn luyện link predictor
    link_predictor.train()
    optimizer.zero_grad()

    # Lấy embeddings cho các cạnh dương (positive edges)
    train_src_embeddings, train_dst_embeddings = get_edge_embeddings(train_edges, embeddings)
    train_src_embeddings = torch.tensor(train_src_embeddings, dtype=torch.float32).to(device)
    train_dst_embeddings = torch.tensor(train_dst_embeddings, dtype=torch.float32).to(device)

    # Lấy embeddings cho các cạnh âm (negative edges)
    neg_src_embeddings, neg_dst_embeddings = get_edge_embeddings(negative_edges, embeddings)
    neg_src_embeddings = torch.tensor(neg_src_embeddings, dtype=torch.float32).to(device)
    neg_dst_embeddings = torch.tensor(neg_dst_embeddings, dtype=torch.float32).to(device)

    # Dự đoán liên kết
    pos_pred = link_predictor(train_src_embeddings, train_dst_embeddings)
    neg_pred = link_predictor(neg_src_embeddings, neg_dst_embeddings)

    # Kết hợp predictions và labels
    predictions = torch.cat([pos_pred, neg_pred], dim=0).squeeze()
    labels = get_link_labels(train_edges, negative_edges)

    # Tính toán loss
    loss = criterion(predictions, labels)
    loss.backward()
    optimizer.step()

    return loss.item()

# 26. Huấn Luyện Link Predictor
print("\nStarting Link Prediction training...")
epochs_lp = 100
best_val_loss_lp = float('inf')

for epoch in range(1, epochs_lp + 1):
    loss = train_link_prediction(model, link_predictor, optimizer_lp, criterion_lp, train_edges, negative_edges, embeddings)
    val_loss = evaluate_link_prediction(model, link_predictor, val_edges, embeddings, criterion_lp)
    print(f'Epoch [{epoch}/{epochs_lp}], Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

    if val_loss < best_val_loss_lp:
        best_val_loss_lp = val_loss
        best_epoch_lp = epoch
        torch.save(link_predictor.state_dict(), 'best_link_prediction_model.pth')

# 27. Đánh Giá Link Predictor Trên Tập Test
print("\nEvaluating Link Predictor on test set...")
link_predictor.load_state_dict(torch.load('best_link_prediction_model.pth'))
test_loss_lp = evaluate_link_prediction(model, link_predictor, test_edges, embeddings, criterion_lp)
print(f'Test Loss: {test_loss_lp:.4f}')

# 28. Thêm Thuộc Tính Link Prediction Vào Edge Table
print("\nAdding Link Prediction results to edge table...")

# Tạo DataFrame cho các cạnh gốc và cạnh dự đoán
existing_edge_df = pd.DataFrame(edge_list_tuples, columns=['source', 'target'])
existing_edge_df['prediction'] = 'existing'

# --- Điều Chỉnh Để Giới Hạn Số Lượng Cạnh Dự Đoán ---
# Thay vì dự đoán tất cả các cạnh không tồn tại, chúng ta đã chọn một tập hợp ngẫu nhiên và giới hạn số lượng cạnh cần dự đoán (100,000)
# Tuy nhiên, chúng ta sẽ chỉ chọn top 100 cạnh dự đoán để tránh sử dụng quá nhiều RAM và tạo thêm các cạnh mới trong Cytoscape.

print("\nCollecting Link Prediction results with limited candidate edges to avoid RAM crash...")
# Giới hạn số lượng cạnh cần dự đoán (ví dụ: 100,000)
max_candidate_edges = 100000

# Tạo danh sách các cạnh không tồn tại
# Chọn ngẫu nhiên một tập hợp các cạnh không tồn tại
candidate_edges = set()
while len(candidate_edges) < max_candidate_edges:
    u = np.random.randint(0, num_nodes)
    v = np.random.randint(0, num_nodes)
    if u != v and (u, v) not in G.edges() and (v, u) not in G.edges():
        candidate_edges.add((u, v))
    if len(candidate_edges) % 10000 == 0 and len(candidate_edges) > 0:
        print(f"{len(candidate_edges)} candidate edges collected...")

candidate_edges = list(candidate_edges)
print(f"Total candidate edges for prediction: {len(candidate_edges)}")

# Lấy embeddings cho các cạnh dự đoán
src_embeddings, dst_embeddings = get_edge_embeddings(candidate_edges, embeddings)
src_embeddings = torch.tensor(src_embeddings, dtype=torch.float32).to(device)
dst_embeddings = torch.tensor(dst_embeddings, dtype=torch.float32).to(device)

# Dự đoán xác suất liên kết
print("\nPredicting link probabilities...")
with torch.no_grad():
    predictions = link_predictor(src_embeddings, dst_embeddings).squeeze().cpu().numpy()

# Chọn các cạnh có xác suất dự đoán cao nhất (ví dụ: top 100)
top_k = 100
top_k_indices = np.argsort(predictions)[-top_k:]
predicted_edges = [candidate_edges[i] for i in top_k_indices]
predicted_scores = predictions[top_k_indices]

print(f"Top {top_k} predicted edges selected.")

# Tạo DataFrame cho các cạnh dự đoán
predicted_edge_df = pd.DataFrame(predicted_edges, columns=['source', 'target'])
predicted_edge_df['prediction'] = 'predicted'
predicted_edge_df['score'] = predicted_scores

# Kết hợp DataFrame cho các cạnh gốc và cạnh dự đoán
combined_edge_df = pd.concat([existing_edge_df, predicted_edge_df[['source', 'target', 'prediction', 'score']]], ignore_index=True)

# 29. Tạo Mạng Lưới Trong Cytoscape với Cả Các Cạnh Dự Đoán
print("\nCreating network in Cytoscape using NetworkX with predicted edges...")
# Tạo đồ thị NetworkX với các node và edge
G_cytoscape = nx.Graph()

# Thêm các node với thuộc tính 'name'
for node in unique_nodes:
    G_cytoscape.add_node(node, name=node)

# Thêm các cạnh hiện có với thuộc tính 'prediction' = 'existing'
for u, v in edge_list:
    G_cytoscape.add_edge(u, v, prediction='existing')

# Thêm các cạnh dự đoán với thuộc tính 'prediction' = 'predicted' và 'score'
for idx, row in predicted_edge_df.iterrows():
    u_idx, v_idx = row['source'], row['target']
    u_node, v_node = idx_to_node[u_idx], idx_to_node[v_idx]
    score = row['score']
    G_cytoscape.add_edge(u_node, v_node, prediction='predicted', score=score)

# Tạo mạng lưới trong Cytoscape từ NetworkX
network_title = 'Drugs Interaction Network'
network_suid = p4c.create_network_from_networkx(G_cytoscape, title=network_title)
print(f"Network created with SUID: {network_suid}")

# Chờ mạng lưới được tạo thành công
print("Waiting for the network to be created in Cytoscape...")
time.sleep(5)  # Chờ 5 giây để đảm bảo mạng lưới đã được tải

# 30. Tải Các Thuộc Tính Embedding vào Cytoscape
print("\nLoading PCA and t-SNE data into Cytoscape...")
# Tải thuộc tính PCA
p4c.load_table_data(
    data=embeddings_df[['name', 'pca_0', 'pca_1']],
    data_key_column='name',
    table='node',
    table_key_column='name',
    network=network_title
)

# Tải thuộc tính t-SNE
p4c.load_table_data(
    data=embeddings_df[['name', 'tsne_0', 'tsne_1', 'tsne_magnitude']],
    data_key_column='name',
    table='node',
    table_key_column='name',
    network=network_title
)

# 31. Định Nghĩa và Áp Dụng Visual Style
print("\nDefining and applying visual styles...")
# Tạo một visual style mới
style_name = "GNN_Homogeneous_Style"
if style_name not in p4c.get_visual_style_names():
    p4c.styles.create_visual_style(style_name)
    print(f"Visual style '{style_name}' created.")
else:
    print(f"Visual style '{style_name}' already exists.")

# Áp dụng visual style
p4c.set_visual_style(style_name)
print(f"Visual style '{style_name}' applied.")

# Thêm nhãn vào DataFrame và tải vào Cytoscape
embeddings_df['label'] = y.cpu().numpy()

# Tải thuộc tính 'label' vào Cytoscape node table
print("\nLoading label data into Cytoscape node table...")
p4c.load_table_data(
    data=embeddings_df[['name', 'label']],
    data_key_column='name',
    table='node',
    table_key_column='name',
    network=network_title  # Sử dụng biến tên mạng lưới đã định nghĩa
)
print("'label' column loaded to Cytoscape node table.")

# Ánh xạ màu sắc cho node dựa trên 'label'
print("Mapping node color based on label...")
# Định nghĩa palette cho hai nhãn bằng cyPalette và chỉ lấy hai màu đầu tiên
palette_colors = p4c.cyPalette(name='set1')[:2]  # Label 0: màu đầu tiên, Label 1: màu thứ hai
palette = ('custom_palette', 'qualitative', lambda n: palette_colors)
print(f"Palette used for mapping: {palette_colors}")

# Tạo color map cho node dựa trên 'label'
label_color_map = p4c.gen_node_color_map(
    table_column='label',
    palette=palette,          # Sử dụng palette đã định nghĩa
    mapping_type='d',         # 'd' cho discrete mapping
    default_color='#CCCCCC',  # Màu xám nhạt mặc định cho các nhãn không xác định
    style_name=style_name,    # Tên style đã tạo trước đó
    network=network_title     # Đảm bảo tên mạng lưới chính xác
)

# Áp dụng color mapping
p4c.set_node_color_mapping(**label_color_map)
print("Node color mapping based on label applied.")

# Ánh xạ 'pca_0' vào kích thước node
print("Mapping PCA component to node size...")
size_map_pca = p4c.gen_node_size_map(
    table_column='pca_0',
    mapping_type='c',          # 'c' cho continuous mapping
    style_name=style_name,
    network=network_title      # Sử dụng biến tên mạng lưới đã định nghĩa
)

p4c.set_node_size_mapping(**size_map_pca)
print("Node size mapping based on 'pca_0' applied.")

# Ánh xạ 'name' vào nhãn node
print("Mapping node labels...")
p4c.style_mappings.set_node_label_mapping('name', style_name=style_name)

# Áp dụng visual style
p4c.set_visual_style(style_name)

# 32. Định Nghĩa và Áp Dụng Edge Style Mapping Generators
print("\nDefining and applying edge style mappings for Link Prediction...")

# Ánh xạ màu sắc cho cạnh dựa trên thuộc tính 'prediction' sử dụng một palette có sẵn
print("Mapping edge color based on prediction status using a predefined palette...")
color_map = p4c.gen_edge_color_map(
    table_column='prediction',
    palette=p4c.palette_color_brewer_q_Set1(),  # Sử dụng hàm palette có sẵn
    mapping_type='d',                          # 'd' cho discrete mapping
    default_color='#000000',                   # Màu đen mặc định
    style_name=style_name,
    network=network_suid
)
p4c.set_edge_color_mapping(**color_map)

# Ánh xạ kiểu đường kẻ cho cạnh dựa trên thuộc tính 'prediction'
print("Mapping edge line style based on prediction status...")
line_style_map = p4c.gen_edge_line_style_map(
    table_column='prediction',
    default_line_style='SOLID',                # Mặc định là SOLID
    style_name=style_name,
    network=network_suid
)
p4c.set_edge_line_style_mapping(**line_style_map)

# Áp dụng visual style
p4c.set_visual_style(style_name)

# Áp dụng layout Force-Directed để sắp xếp mạng lưới
print("Applying Force-Directed layout...")
p4c.layout_network('force-directed', network=network_suid)


DataFrame head:
    source   target
0  DB00862  DB00966
1  DB00575  DB00806
2  DB01242  DB08893
3  DB01151  DB08883
4  DB01235  DB01275
Total nodes: 1514
Number of nodes: 1514
Number of edges: 48514
Setting up training...
Starting training...
Epoch: 020, Loss: 0.6756, Train Acc: 0.5534, Val Acc: 0.4670
Epoch: 040, Loss: 0.6137, Train Acc: 0.6223, Val Acc: 0.4361
Epoch: 060, Loss: 0.5319, Train Acc: 0.7092, Val Acc: 0.4714
Epoch: 080, Loss: 0.4703, Train Acc: 0.7658, Val Acc: 0.4890
Epoch: 100, Loss: 0.4381, Train Acc: 0.7875, Val Acc: 0.5022
Epoch: 120, Loss: 0.3979, Train Acc: 0.7970, Val Acc: 0.5198
Epoch: 140, Loss: 0.3964, Train Acc: 0.7885, Val Acc: 0.4934
Epoch: 160, Loss: 0.3923, Train Acc: 0.8045, Val Acc: 0.5022
Epoch: 180, Loss: 0.3751, Train Acc: 0.8291, Val Acc: 0.5154
Epoch: 200, Loss: 0.4389, Train Acc: 0.8036, Val Acc: 0.4978
Epoch: 220, Loss: 0.3246, Train Acc: 0.8527, Val Acc: 0.4758
Epoch: 240, Loss: 0.3075, Train Acc: 0.8489, Val Acc: 0.5022
Epoch: 260, Loss: 0.3239,

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_31392\3381416346.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pt'))
c:\User


Converting edge_list_tuples to node indices...

Splitting edges into train/validation/test sets for Link Prediction...

Creating negative edges for training...
